<a href="https://colab.research.google.com/github/rl-cyber/User-Engaged-Network-Diagnosis/blob/main/Reproduce_cellularlint_codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/CellularLint/cellularlint-codes.git


Cloning into 'cellularlint-codes'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 127 (delta 66), reused 65 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (127/127), 2.17 MiB | 5.36 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [2]:
%cd cellularlint-codes

/content/cellularlint-codes


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/Pretrained_Models.zip ./
!unzip ./Pretrained_Models.zip -d Pretrained_Models/

Archive:  ./Pretrained_Models.zip
 extracting: Pretrained_Models/roberta_weights.zip  
 extracting: Pretrained_Models/bert_weights.zip  
 extracting: Pretrained_Models/xlnet_weights.zip  


In [5]:
!chmod 700 unpack.sh
!./unpack.sh

Unzipped bert_weights.zip into bert_weights
Unzipped roberta_weights.zip into roberta_weights
Unzipped xlnet_weights.zip into xlnet_weights


In [6]:
!python3 tokenizer_and_sim_matrix.py 4G
!python3 tokenizer_and_sim_matrix.py 5G

tokenizer_config.json: 100% 1.19k/1.19k [00:00<00:00, 9.15MB/s]
config.json: 100% 594/594 [00:00<00:00, 4.67MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 2.91MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 5.92MB/s]
special_tokens_map.json: 100% 239/239 [00:00<00:00, 2.39MB/s]
2025-02-12 00:59:04.565680: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739321944.829761    1195 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739321944.906820    1195 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 00:59:05.493551: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in pe

In [16]:
from transformers import BertModel, BertTokenizerFast

MODEL_SAVE_PATH = "/content/cellularlint-codes/saved_models/bert"

# Download and save BERT model and tokenizer
model = BertModel.from_pretrained("bert-base-uncased")
model.save_pretrained(MODEL_SAVE_PATH)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.save_pretrained(MODEL_SAVE_PATH)

print("BERT model and tokenizer saved at:", MODEL_SAVE_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BERT model and tokenizer saved at: /content/cellularlint-codes/saved_models/bert


In [21]:
import pandas as pd
import os
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, BertConfig, BertTokenizerFast
from tokenizers import BertWordPieceTokenizer
from torch import nn
np.random.seed(321)

BertModel.from_pretrained("./Pretrained Models/bert_weights/")

ROOT_PATH = "/content/cellularlint-codes"

# Set paths for dataset and models
#DATA_PATH = os.path.join(ROOT_PATH, "Data", "SNLI")
#MODEL_PATH = os.path.join(ROOT_PATH, "Pretrained Models", "bert_weights")

# Ensure paths exist
#os.makedirs(DATA_PATH, exist_ok=True)
#os.makedirs(MODEL_PATH, exist_ok=True)

#ROOT_PATH = "./cellularlint-codes"
#DATA_PATH = "./Data/SNLI/"
#PRETRAINED_PATH = "./Pretrained Models/bert_weights/"
#SAVE_MODEL_AT = "./saved_models/bert"
#PRETRAINED_TOKENIZER = "./saved_models/bert"
#MODEL_PATH = None


DATA_PATH = os.path.join(ROOT_PATH, "Data/SNLI")
PRETRAINED_PATH = os.path.join(ROOT_PATH, "Pretrained Models/bert_weights")
SAVE_MODEL_AT = os.path.join(ROOT_PATH, "saved_models/bert")
PRETRAINED_TOKENIZER = SAVE_MODEL_AT  # Tokenizer is usually saved in the same model folder
MODEL_PATH = None


df_train = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_train.csv"))
df_dev = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_dev.csv"))
df_test = pd.read_csv(os.path.join(DATA_PATH,"snli_1.0_test.csv"))


df_train = df_train.iloc[:500]
df_dev = df_train.copy().iloc[:100]


tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_TOKENIZER))
tokenizer = BertTokenizerFast.from_pretrained(os.path.join(ROOT_PATH,PRETRAINED_PATH))
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


labels = {'contradiction':1,
          'entailment':0,
          'neutral':2,
          }
NUM_LABELS = len(labels)

def bert_encoding(sequence):
    return tokenizer.encode(sequence, add_special_tokens = False)
def str_to_int_list(data):
    return list(map(int, data))

df_train['token_type'] = df_train['token_type'].str.split()
df_train['token_type'] = df_train['token_type'].apply(str_to_int_list)

df_dev['token_type'] = df_dev['token_type'].str.split()
df_dev['token_type'] = df_dev['token_type'].apply(str_to_int_list)

df_test['token_type'] = df_test['token_type'].str.split()
df_test['token_type'] = df_test['token_type'].apply(str_to_int_list)



df_train['attention_mask'] = df_train['attention_mask'].str.split()
df_train['attention_mask'] = df_train['attention_mask'].apply(str_to_int_list)

df_dev['attention_mask'] = df_dev['attention_mask'].str.split()
df_dev['attention_mask'] = df_dev['attention_mask'].apply(str_to_int_list)

df_test['attention_mask'] = df_test['attention_mask'].str.split()
df_test['attention_mask'] = df_test['attention_mask'].apply(str_to_int_list)


df_train['input_ids'] = df_train['sequence'].apply(bert_encoding)
df_dev['input_ids'] = df_dev['sequence'].apply(bert_encoding)
df_test['input_ids'] = df_test['sequence'].apply(bert_encoding)


max_length = 512
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.texts = []
        self.labels = [labels[label] for label in df['gold_label']]
        for _, row in df.iterrows():
            token_type_ids = row['token_type']
            token_type_ids += [0] * (max_length - len(token_type_ids))
            attention_mask = row['attention_mask']
            attention_mask += [0] * (max_length - len(attention_mask))
            input_ids = tokenizer.encode(
                row['sequence'],
                add_special_tokens=False,
                padding='max_length',
                max_length=max_length,
                truncation=True,
                return_tensors="pt"
            )
            datadict = {
                'input_ids': input_ids.squeeze(0),
                'token_type_ids': torch.tensor(token_type_ids),
                'attention_mask': torch.tensor(attention_mask)
            }
            self.texts.append(datadict)

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        batch_text = self.texts[idx]
        batch_y = self.labels[idx]
        return batch_text, batch_y


class BertClassifier(nn.Module):

    def __init__(self, load_path = None, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(os.path.join(ROOT_PATH, load_path))
        self.bert = BertModel.from_pretrained(load_path)

        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, NUM_LABELS)

        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_id, mask, token_type_id):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask = mask, token_type_ids = token_type_id,return_dict = False)

        dropout_output = self.dropout(pooled_output)

        linear_output = self.linear(dropout_output)

        final_layer = self.softmax(linear_output)

        return final_layer

    def save(self, save_dir, tokenizer, model_name = "model_bert.pt"):

        os.makedirs(save_dir, exist_ok=True)
        # Save model weights
        model_path = os.path.join(save_dir, model_name)
        torch.save(self.state_dict(), model_path)
        self.bert.save_pretrained(save_dir)

        # Save tokenizer
        tokenizer.save_pretrained(save_dir)

        # Save other related information
        config_path = os.path.join(save_dir, "config.json")
        self.bert.config.to_json_file(config_path)

    def load(self, load_dir, is_eval = True, model_name = "model_bert.pt"):
        # Load tokenizer
        tokenizer = BertTokenizer.from_pretrained(load_dir)

        # Load other related information
        config_path = os.path.join(load_dir, "config.json")
        config = BertConfig.from_json_file(config_path)
        self.bert = BertModel(config)
        if is_eval:
            self.bert.eval()  # Set to evaluation mode

        # Load model weights
        model_path = os.path.join(load_dir, model_name)
        self.load_state_dict(torch.load(model_path))

        # Update the tokenizer
        self.bert.resize_token_embeddings(len(tokenizer))


from tqdm import tqdm
learning_rates = [1e-5] #[5e-6, 1e-5, 2e-5, 3e-5, 5e-5]
batch_sizes = [8] #[16, 24, 32, 40]

def train(model, train_data, val_data, learning_rate, epochs, batch_size):
    #out_file.write(f"-------------Starting with LR = {learning_rate} and BS = {batch_size}-----------------\n")
    best_acc_val = -99999

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
                #print(train_input)
                train_label = train_label.to(device)

                input_id = train_input['input_ids'].squeeze(1).to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                token_type_id = train_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                print(f'training logits: {logits}')
                print("training outputs")
                print(output)
                optimizer.zero_grad()

                batch_loss = criterion(logits, train_label.long())
                total_loss_train += batch_loss.item()
                print(f'prediction: {torch.argmax(logits, dim=1)}')
                print(f"train labels: {train_label}")
                acc = (torch.argmax(logits, dim=1) == train_label).sum().item()
                total_acc_train += acc


                batch_loss.backward()
                optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                mask = val_input['attention_mask'].squeeze(1).to(device)
                token_type_id = val_input['token_type_ids'].squeeze(1).to(device)

                output = model(input_id, mask, token_type_id)
                #logits = output.logits #For BertForSequenceClassification
                logits = output
                print(val_label)
                print(f'validating logits: {logits}')
                print(f'prediction: {torch.argmax(logits, dim=1)}')

                batch_loss = criterion(logits, val_label.long())
                total_loss_val += batch_loss.item()
                print(output)
                acc = (torch.argmax(logits, dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')

        if(total_acc_val / len(val_data) > best_acc_val):
            best_acc_val = total_acc_val / len(val_data)
            model.bert.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))
            tokenizer.save_pretrained(os.path.join(ROOT_PATH,"saved_models/finetune_p1/bert/"))

            model.save(save_dir = os.path.join(ROOT_PATH, SAVE_MODEL_AT), tokenizer = tokenizer)
            print("Found a better model")
            model.save(save_dir = SAVE_MODEL_AT, tokenizer = tokenizer)

EPOCHS = 8
for LR in learning_rates:
    for bs in batch_sizes:
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = NUM_LABELS)
        model = BertClassifier(load_path = PRETRAINED_PATH)
        train(model, df_train, df_dev, LR, EPOCHS, bs)



Some weights of BertModel were not initialized from the model checkpoint at ./Pretrained Models/bert_weights/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at /content/cellularlint-codes/Pretrained Models/bert_weights and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at 

training logits: tensor([[0.2372, 0.2502, 0.5126],
        [0.1687, 0.2285, 0.6028],
        [0.2515, 0.1548, 0.5937],
        [0.0563, 0.1504, 0.7933],
        [0.1208, 0.1972, 0.6820],
        [0.3632, 0.1407, 0.4961],
        [0.1546, 0.1917, 0.6537],
        [0.4178, 0.2227, 0.3595]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.2372, 0.2502, 0.5126],
        [0.1687, 0.2285, 0.6028],
        [0.2515, 0.1548, 0.5937],
        [0.0563, 0.1504, 0.7933],
        [0.1208, 0.1972, 0.6820],
        [0.3632, 0.1407, 0.4961],
        [0.1546, 0.1917, 0.6537],
        [0.4178, 0.2227, 0.3595]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([2, 2, 2, 2, 2, 2, 2, 0], device='cuda:0')
train labels: tensor([1, 2, 1, 2, 2, 2, 2, 1], device='cuda:0')


  2%|▏         | 1/63 [00:00<00:46,  1.34it/s]

training logits: tensor([[0.3740, 0.1057, 0.5203],
        [0.2447, 0.1343, 0.6210],
        [0.2817, 0.3015, 0.4168],
        [0.2434, 0.0689, 0.6877],
        [0.3867, 0.1363, 0.4769],
        [0.1566, 0.0598, 0.7836],
        [0.2240, 0.1736, 0.6024],
        [0.2571, 0.1343, 0.6086]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.3740, 0.1057, 0.5203],
        [0.2447, 0.1343, 0.6210],
        [0.2817, 0.3015, 0.4168],
        [0.2434, 0.0689, 0.6877],
        [0.3867, 0.1363, 0.4769],
        [0.1566, 0.0598, 0.7836],
        [0.2240, 0.1736, 0.6024],
        [0.2571, 0.1343, 0.6086]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
train labels: tensor([2, 1, 1, 1, 1, 1, 1, 2], device='cuda:0')


  3%|▎         | 2/63 [00:01<00:42,  1.43it/s]

...

training logits: tensor([[0.0266, 0.0109, 0.9625],
        [0.9938, 0.0017, 0.0044],
        [0.9901, 0.0027, 0.0072],
        [0.9934, 0.0027, 0.0039],
        [0.0223, 0.9465, 0.0312],
        [0.7913, 0.1723, 0.0363],
        [0.0206, 0.9691, 0.0103],
        [0.4650, 0.2097, 0.3253]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.0266, 0.0109, 0.9625],
        [0.9938, 0.0017, 0.0044],
        [0.9901, 0.0027, 0.0072],
        [0.9934, 0.0027, 0.0039],
        [0.0223, 0.9465, 0.0312],
        [0.7913, 0.1723, 0.0363],
        [0.0206, 0.9691, 0.0103],
        [0.4650, 0.2097, 0.3253]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([2, 0, 0, 0, 1, 0, 1, 0], device='cuda:0')
train labels: tensor([2, 0, 0, 0, 1, 2, 1, 1], device='cuda:0')


 98%|█████████▊| 62/63 [00:45<00:00,  1.37it/s]

training logits: tensor([[0.9726, 0.0120, 0.0154],
        [0.0608, 0.0460, 0.8932],
        [0.9842, 0.0074, 0.0084],
        [0.0597, 0.0277, 0.9125]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
training outputs
tensor([[0.9726, 0.0120, 0.0154],
        [0.0608, 0.0460, 0.8932],
        [0.9842, 0.0074, 0.0084],
        [0.0597, 0.0277, 0.9125]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
prediction: tensor([0, 2, 0, 2], device='cuda:0')
train labels: tensor([0, 2, 2, 2], device='cuda:0')


100%|██████████| 63/63 [00:45<00:00,  1.38it/s]


tensor([2, 1, 0, 2, 0, 1, 1, 0], device='cuda:0')
validating logits: tensor([[0.0403, 0.0348, 0.9248],
        [0.0141, 0.9641, 0.0218],
        [0.9938, 0.0032, 0.0030],
        [0.0407, 0.0387, 0.9206],
        [0.9920, 0.0021, 0.0058],
        [0.0112, 0.9813, 0.0076],
        [0.0222, 0.9169, 0.0609],
        [0.9926, 0.0029, 0.0045]], device='cuda:0')
prediction: tensor([2, 1, 0, 2, 0, 1, 1, 0], device='cuda:0')
tensor([[0.0403, 0.0348, 0.9248],
        [0.0141, 0.9641, 0.0218],
        [0.9938, 0.0032, 0.0030],
        [0.0407, 0.0387, 0.9206],
        [0.9920, 0.0021, 0.0058],
        [0.0112, 0.9813, 0.0076],
        [0.0222, 0.9169, 0.0609],
        [0.9926, 0.0029, 0.0045]], device='cuda:0')
tensor([2, 2, 1, 2, 2, 1, 0, 2], device='cuda:0')
validating logits: tensor([[0.0096, 0.0404, 0.9500],
        [0.0894, 0.0590, 0.8516],
        [0.0198, 0.9700, 0.0102],
        [0.0682, 0.0364, 0.8954],
        [0.0299, 0.0334, 0.9367],
        [0.0079, 0.9844, 0.0077],
        [0.9903,